# CSCI 4253 / 5253 - Lab #3 - Patent Problem with SQL - SOLUTION
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

In this assignment, we're going to solve a problem for which you'll also see the solution in Hadoop and then implement a PySpark solution. We have two databases
* One contains information about patents
* One contains information about patent citations (one patent citing the work of another)
The problem we're going to solve is augmenting the original patent data to include the number of *co-state citations*. In other words, if patent X was issued to someone in Colorado and patent Y was also issued to someone in Colorado and X cites Y, then this is a co-state citation.

The easiest way to solve this is to build an intermediary product from the patent citations table. That table contains `CITING` and `CITED` columns; you would augment it by adding `CITING_STATE` and `CITED_STATE`. Then, it becomes fairly simple to filter out all the cases where those states don't match. You can then use an sql `GROUP BY` to `COUNT(*)` all the co-state citations for a given patent and then join that with the original patents table resulting in an augmented table.

The final results for the first 13 words, sorted in descending order by co-state citations looks like the following
![this final output](final-output.png)

The challenge is going to be that I want you do to this *a single SQL query*. We're assuming you've learned some SQL in a previous life and if not, [now is a great time to learn](https://www.sqlitetutorial.net/). Even if you've done basic SQL, you'll probably need to review [using `select` in a where-clause or using multiple joins](https://dba.stackexchange.com/questions/33553/using-select-in-the-where-clause-of-another-select).


## Logistics

We're going to be using the SQLite3 system which runs entirely from a file (no server needed). The `Makefile` contains commands to download the raw data as ZIP files.

We can run shell commands in our notebook using [builtin "magic" commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html). You need to run this at least once prior to starting the lab to make certain you have the files and have created the `patents.sq3` database file. It will take a few minutes to complete & you should see that the `patents.sq3` file is about 645MBytes in size.

In [1]:
# Temporary fix to CSEL version conflict issue
%pip install --upgrade --user ipython-sql==0.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [ipython-sql] [sqlalchemy]
Note: you may need to restart the kernel to use updated packages.


In [5]:
%%bash
make
# rm patents.sq3
# zcat < acite75_99.zip | sqlite3 patents.sq3 ".mode csv" ".import /dev/stdin citations"
# zcat < apat63_99.zip | sqlite3 patents.sq3 ".mode csv" ".import /dev/stdin patents"
# ls -l patents.sq3

rm -f patents.sq3

unzip -p acite75_99.zip | sqlite3 patents.sq3 ".mode csv" ".import /dev/stdin citations"
unzip -p apat63_99.zip  | sqlite3 patents.sq3 ".mode csv" ".import /dev/stdin patents"

ls -lh patents.sq3


make: Nothing to be done for `all'.
-rw-r--r--@ 1 kendallahern  staff   620M Sep 11 15:41 patents.sq3


We'll use another "magic" to run SQL queries in notebook cells. The following will load the SQL extension and connect to the `patents.sq3` file.

In [12]:
%load_ext sql
%sql sqlite:///patents.sq3
%config SqlMagic.style = '_DEPRECATED_DEFAULT'


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Following this, we can run individual SQL queries and see the result by putting `%%sql` at the front of a cell. If you don't have that, you'll be running Python code.

So, for example, we can examine our two raw database tables.

In [14]:
%%sql
select * from patents limit 5;

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD
3070801,1963,1096,,BE,,,1,,269,6,69,,1,,0,,,,,,,
3070802,1963,1096,,US,TX,,1,,2,6,63,,0,,,,,,,,,
3070803,1963,1096,,US,IL,,1,,2,6,63,,9,,0.3704,,,,,,,
3070804,1963,1096,,US,OH,,1,,2,6,63,,3,,0.6667,,,,,,,
3070805,1963,1096,,US,CA,,1,,2,6,63,,1,,0,,,,,,,


In [15]:
%%sql
select * from citations limit 5;

 * sqlite:///patents.sq3
Done.


CITING,CITED
3858241,956203
3858241,1324234
3858241,3398406
3858241,3557384
3858241,3634889


If you want to create indexes over various fields, go ahead. It shouldn't affect the correctness of your results but may affect the performance.

## Steps to the full solution

In order to determine when a *cited* patent and a *citing* patent are from the same state,
we're going to need to produce a series of tables that combine information from the citations and the patents tables.

We can use a simple left-outer join (or just **JOIN**) to get the information for one or the other column in the citations table. For example, we can determine the state for *cited* patents using this join:

In [16]:
%%sql
SELECT CITED, patents.POSTATE as CITED_POSTATE, CITING
FROM citations JOIN patents
WHERE CITED==PATENT limit 5;

 * sqlite:///patents.sq3
Done.


CITED,CITED_POSTATE,CITING
3398406,FL,3858241
3557384,MA,3858241
3634889,OH,3858241
3319261,OH,3858242
3668705,WI,3858242


Then, you'll need to do the same for the `CITING` column as well. As mentioned earlier, you may want to review [using `select` in a where-clause or using multiple joins](https://dba.stackexchange.com/questions/33553/using-select-in-the-where-clause-of-another-select).

## Your solution

Eneter your solution as a single SQL query below:

In [17]:
# Your solution should be in the last cell
"""
Start from patents table.
Join twice - once for citing patent and once for cited
    Want the states
Filter rows where same state
Count co-state citations for each patent
Join these results back into original patents table
"""

'\nStart from patents table.\nJoin twice - once for citing patent and once for cited\n    Want the states\nFilter rows where same state\nCount co-state citations for each patent\nJoin these results back into original patents table\n'

In [18]:
#Steps/debug
#add citing patents state to citation

In [19]:
%%sql
SELECT c.CITING, 
    c.CITED, 
    p.POSTATE AS CITING_POSTSTATE
FROM citations AS c
JOIN patents AS p
    ON c.CITING=p.PATENT
LIMIT 5
        

 * sqlite:///patents.sq3
Done.


CITING,CITED,CITING_POSTSTATE
3858241,956203,MA
3858241,1324234,MA
3858241,3398406,MA
3858241,3557384,MA
3858241,3634889,MA


In [20]:
#add cited patents state

In [21]:
%%sql
SELECT c.CITING, 
    c.CITED, 
    p.POSTATE AS CITED_POSTSTATE
FROM citations AS c
JOIN patents AS p
    ON c.CITED=p.PATENT
LIMIT 5
        

 * sqlite:///patents.sq3
Done.


CITING,CITED,CITED_POSTSTATE
3858241,3398406,FL
3858241,3557384,MA
3858241,3634889,OH
3858242,3319261,OH
3858242,3668705,WI


In [22]:
#Combine

In [23]:
%%sql
SELECT c.CITING, 
    pciting.POSTATE AS CITING_POSTATE,
    c.CITED, 
    pcited.POSTATE AS CITED_POSTATE
FROM citations AS c
JOIN patents AS pciting
    ON c.CITING=pciting.PATENT
JOIN patents AS pcited
    ON c.CITED=pcited.PATENT
LIMIT 5
        

 * sqlite:///patents.sq3
Done.


CITING,CITING_POSTATE,CITED,CITED_POSTATE
3858241,MA,3398406,FL
3858241,MA,3557384,MA
3858241,MA,3634889,OH
3858242,MI,3319261,OH
3858242,MI,3668705,WI


In [24]:
#Filter co-state

In [25]:
%%sql
SELECT c.CITING, 
    pciting.POSTATE AS CITING_POSTATE,
    c.CITED, 
    pcited.POSTATE AS CITED_POSTATE
FROM citations AS c
JOIN patents AS pciting
    ON c.CITING=pciting.PATENT
JOIN patents AS pcited
    ON c.CITED=pcited.PATENT
WHERE pciting.POSTATE=pcited.POSTATE
LIMIT 5

 * sqlite:///patents.sq3
Done.


CITING,CITING_POSTATE,CITED,CITED_POSTATE
3858241,MA,3557384,MA
3858245,NY,3755824,NY
3858247,CA,3621837,CA
3858247,CA,3694819,CA
3858249,TX,3418664,TX


In [26]:
#eliminate null cases

In [27]:
%%sql
SELECT c.CITING, 
    pciting.POSTATE AS CITING_POSTATE,
    c.CITED, 
    pcited.POSTATE AS CITED_POSTATE
FROM citations AS c
JOIN patents AS pciting
    ON c.CITING=pciting.PATENT
JOIN patents AS pcited
    ON c.CITED=pcited.PATENT
WHERE pciting.POSTATE=pcited.POSTATE
    AND pciting.POSTATE IS NOT NULL
    AND pcited.POSTATE IS NOT NULL
    AND pciting.POSTATE <>''
    AND pcited.POSTATE <>''
LIMIT 5

 * sqlite:///patents.sq3
Done.


CITING,CITING_POSTATE,CITED,CITED_POSTATE
3858241,MA,3557384,MA
3858245,NY,3755824,NY
3858247,CA,3621837,CA
3858247,CA,3694819,CA
3858249,TX,3418664,TX


In [28]:
#count co state

In [29]:
%%sql
SELECT c.CITED AS patent_id,
    COUNT(*) AS co_state_count
FROM citations AS c
JOIN patents AS pciting
    ON c.CITING=pciting.PATENT
JOIN patents AS pcited
    ON c.CITED=pcited.PATENT
WHERE pciting.POSTATE=pcited.POSTATE
    AND pciting.POSTATE IS NOT NULL
    AND pcited.POSTATE IS NOT NULL
    AND pciting.POSTATE <>''
    AND pcited.POSTATE <>''
GROUP BY c.CITED
ORDER BY co_state_count DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


patent_id,co_state_count
3845770,268
5111638,247
4733521,224
4773182,215
4683195,213
4418068,208
4133814,207
3916899,198
4683202,188
4216620,184


In [30]:
#Join

In [31]:
%%sql
SELECT p.PATENT,
    p.POSTATE,
    IFNULL(cs.co_state_count, 0) AS co_state_citations
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
        COUNT(*) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
        ON c.CITING=pciting.PATENT
    JOIN patents AS pcited
        ON c.CITED=pcited.PATENT
    WHERE pciting.POSTATE=pcited.POSTATE
        AND pciting.POSTATE IS NOT NULL
        AND pcited.POSTATE IS NOT NULL
        AND pciting.POSTATE <>''
        AND pcited.POSTATE <>''
    GROUP BY c.CITED
) AS cs
ON p.PATENT=cs.patent_id
ORDER BY co_state_count DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


PATENT,POSTATE,co_state_citations
3845770,CA,268
5111638,IL,247
4733521,IL,224
4773182,IL,215
4683195,CA,213
4418068,IN,208
4133814,IN,207
3916899,CA,198
4683202,CA,188
4216620,IL,184


In [32]:
#Only had 3 columns, want whole thing

In [33]:
%%sql
SELECT p.*,
    IFNULL(cs.co_state_count, 0) AS co_state_citations
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
        COUNT(*) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
        ON c.CITING=pciting.PATENT
    JOIN patents AS pcited
        ON c.CITED=pcited.PATENT
    WHERE pciting.POSTATE=pcited.POSTATE
        AND pciting.POSTATE IS NOT NULL
        AND pcited.POSTATE IS NOT NULL
        AND pciting.POSTATE <>''
        AND pcited.POSTATE <>''
    GROUP BY c.CITED
) AS cs
ON p.PATENT=cs.patent_id
ORDER BY co_state_count DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,co_state_citations
3845770,1974,5422,1972,US,CA,22715,2,,424,3,31,,339,,0.5152,,17.0118,,,,0.7818,0.7611,268
5111638,1992,11820,1991,US,IL,251075,2,16,53,6,68,39,254,0.4872,0.6286,0.7867,4.3307,32.0256,0.1818,0.0513,0.1633,0.1614,247
4733521,1988,10315,1986,US,IL,251075,2,16,53,6,68,24,236,0.5,0.7189,0.75,8.6737,30.9167,0,0,0.2684,0.2627,224
4773182,1988,10497,1987,US,IL,251075,2,10,47,6,61,27,225,0.5556,0.7597,0.8178,7.6444,29.4444,0.1667,0.037,0.2968,0.2889,215
4683195,1987,10070,1986,US,CA,97235,2,26,435,3,33,1,631,1,0.3464,0,7.8494,4,0,0,0.0183,0.0174,213
4418068,1983,8733,1981,US,IN,170335,2,62,514,3,31,7,225,1,0.3028,0.6939,14.1467,5.7143,0.4286,0.4286,0.9372,0.9289,208
4133814,1979,6948,1976,US,IN,170335,2,10,548,1,14,4,219,1,0.316,0.5,18.9543,2.5,0,0,0.9495,0.9452,207
3916899,1975,5786,1974,US,CA,22715,2,36,424,3,31,1,231,1,0.3987,0,15.684,1,1,1,0.8438,0.8182,198
4683202,1987,10070,1985,US,CA,97235,2,21,435,3,33,0,605,,0.3529,,8.8843,,,,0.0157,0.0149,188
4216620,1980,7529,1976,US,IL,251075,2,5,47,6,61,7,197,0,0.7048,,18.269,51.5714,0,0,0.2957,0.2792,184


In [34]:
#same as above but bettter matches screenshot output

In [35]:
%%sql
SELECT p.*,
    IFNULL(cs.co_state_count, 0) AS CO_CITED_COUNT
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
        COUNT(*) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
        ON c.CITING=pciting.PATENT
    JOIN patents AS pcited
        ON c.CITED=pcited.PATENT
    WHERE pciting.POSTATE=pcited.POSTATE
        AND pciting.POSTATE IS NOT NULL
        AND pcited.POSTATE IS NOT NULL
        AND pciting.POSTATE <>''
        AND pcited.POSTATE <>''
    GROUP BY c.CITED
) AS cs
ON p.PATENT=cs.patent_id
ORDER BY CO_CITED_COUNT DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,CO_CITED_COUNT
3845770,1974,5422,1972,US,CA,22715,2,,424,3,31,,339,,0.5152,,17.0118,,,,0.7818,0.7611,268
5111638,1992,11820,1991,US,IL,251075,2,16,53,6,68,39,254,0.4872,0.6286,0.7867,4.3307,32.0256,0.1818,0.0513,0.1633,0.1614,247
4733521,1988,10315,1986,US,IL,251075,2,16,53,6,68,24,236,0.5,0.7189,0.75,8.6737,30.9167,0,0,0.2684,0.2627,224
4773182,1988,10497,1987,US,IL,251075,2,10,47,6,61,27,225,0.5556,0.7597,0.8178,7.6444,29.4444,0.1667,0.037,0.2968,0.2889,215
4683195,1987,10070,1986,US,CA,97235,2,26,435,3,33,1,631,1,0.3464,0,7.8494,4,0,0,0.0183,0.0174,213
4418068,1983,8733,1981,US,IN,170335,2,62,514,3,31,7,225,1,0.3028,0.6939,14.1467,5.7143,0.4286,0.4286,0.9372,0.9289,208
4133814,1979,6948,1976,US,IN,170335,2,10,548,1,14,4,219,1,0.316,0.5,18.9543,2.5,0,0,0.9495,0.9452,207
3916899,1975,5786,1974,US,CA,22715,2,36,424,3,31,1,231,1,0.3987,0,15.684,1,1,1,0.8438,0.8182,198
4683202,1987,10070,1985,US,CA,97235,2,21,435,3,33,0,605,,0.3529,,8.8843,,,,0.0157,0.0149,188
4216620,1980,7529,1976,US,IL,251075,2,5,47,6,61,7,197,0,0.7048,,18.269,51.5714,0,0,0.2957,0.2792,184


In [36]:
#numbers are off, try to remove the not null section

In [37]:
%%sql
SELECT p.*,
    IFNULL(cs.co_state_count, 0) AS CO_CITED_COUNT
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
        COUNT(*) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
        ON c.CITING=pciting.PATENT
    JOIN patents AS pcited
        ON c.CITED=pcited.PATENT
    WHERE pciting.POSTATE=pcited.POSTATE
    GROUP BY c.CITED
) AS cs
ON p.PATENT=cs.patent_id
ORDER BY CO_CITED_COUNT DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,CO_CITED_COUNT
4723129,1988,10259,1986,JP,,87490,3,9,347,2,23,14,779,0.9286,0.4728,0.7811,7.6314,15.5,0,0,0.9379,0.9307,757
4740796,1988,10343,1986,JP,,87490,3,10,347,2,23,14,678,0.9286,0.4826,0.7811,7.753,15.5,0,0,0.9852,0.9808,673
4463359,1984,8978,1980,JP,,87490,3,51,347,2,23,9,716,1,0.3852,0.3704,13.4888,6.5556,0.4444,0.4444,0.899,0.8953,655
4345262,1982,8264,1980,JP,,87490,3,15,347,2,23,3,658,0.6667,0.3748,0,13.6159,10.6667,0.5,0.3333,0.9497,0.9468,637
4313124,1982,8061,1980,JP,,87490,3,7,347,2,23,2,633,1,0.3855,0,13.7116,1,1,1,0.9635,0.9589,620
4558333,1985,9475,1982,JP,,87490,3,14,347,2,23,6,654,1,0.4002,0.5,11.7914,6.8333,0.6,0.5,0.9264,0.9235,611
4459600,1984,8957,1981,JP,,87490,3,2,347,2,23,6,613,0.8333,0.3933,0.32,12.8151,12.5,0.25,0.1667,0.9918,0.9886,609
3845770,1974,5422,1972,US,CA,22715,2,,424,3,31,,339,,0.5152,,17.0118,,,,0.7818,0.7611,268
5111638,1992,11820,1991,US,IL,251075,2,16,53,6,68,39,254,0.4872,0.6286,0.7867,4.3307,32.0256,0.1818,0.0513,0.1633,0.1614,247
4258264,1981,7753,1979,JP,,211155,3,4,250,4,44,2,251,1,0.4644,0,9.4422,5.5,0,0,0.9194,0.9084,238


In [38]:
%%sql
SELECT c.CITING, pciting.POSTATE, pcited.POSTATE
FROM citations AS c
JOIN patents AS pciting ON c.CITING = pciting.PATENT
JOIN patents AS pcited ON c.CITED = pcited.PATENT
WHERE c.CITED = 5663077;


 * sqlite:///patents.sq3
Done.


CITING,POSTATE,POSTATE_1
5827772,,
5840600,,
5844640,,
5858823,,
5869362,,
5869363,,
5879960,,
5879974,,
5879977,,
5886366,,


In [39]:
#Final Solution:

In [40]:
%%sql
SELECT p.*,
    IFNULL(cs.co_state_count, 0) AS CO_CITED_COUNT
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
        COUNT(*) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
        ON c.CITING=pciting.PATENT
    JOIN patents AS pcited
        ON c.CITED=pcited.PATENT
    WHERE pciting.POSTATE=pcited.POSTATE
    GROUP BY c.CITED
) AS cs
ON p.PATENT=cs.patent_id
ORDER BY CO_CITED_COUNT DESC
LIMIT 13

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,CO_CITED_COUNT
4723129,1988,10259,1986,JP,,87490,3,9,347,2,23,14,779,0.9286,0.4728,0.7811,7.6314,15.5,0,0,0.9379,0.9307,757
4740796,1988,10343,1986,JP,,87490,3,10,347,2,23,14,678,0.9286,0.4826,0.7811,7.753,15.5,0,0,0.9852,0.9808,673
4463359,1984,8978,1980,JP,,87490,3,51,347,2,23,9,716,1,0.3852,0.3704,13.4888,6.5556,0.4444,0.4444,0.899,0.8953,655
4345262,1982,8264,1980,JP,,87490,3,15,347,2,23,3,658,0.6667,0.3748,0,13.6159,10.6667,0.5,0.3333,0.9497,0.9468,637
4313124,1982,8061,1980,JP,,87490,3,7,347,2,23,2,633,1,0.3855,0,13.7116,1,1,1,0.9635,0.9589,620
4558333,1985,9475,1982,JP,,87490,3,14,347,2,23,6,654,1,0.4002,0.5,11.7914,6.8333,0.6,0.5,0.9264,0.9235,611
4459600,1984,8957,1981,JP,,87490,3,2,347,2,23,6,613,0.8333,0.3933,0.32,12.8151,12.5,0.25,0.1667,0.9918,0.9886,609
3845770,1974,5422,1972,US,CA,22715,2,,424,3,31,,339,,0.5152,,17.0118,,,,0.7818,0.7611,268
5111638,1992,11820,1991,US,IL,251075,2,16,53,6,68,39,254,0.4872,0.6286,0.7867,4.3307,32.0256,0.1818,0.0513,0.1633,0.1614,247
4258264,1981,7753,1979,JP,,211155,3,4,250,4,44,2,251,1,0.4644,0,9.4422,5.5,0,0,0.9194,0.9084,238


In [41]:
%%sql
SELECT p.*,
       IFNULL(cs.co_state_count, 0) AS co_state_citations
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
           COUNT(DISTINCT c.CITING) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
      ON c.CITING = pciting.PATENT
    JOIN patents AS pcited
      ON c.CITED = pcited.PATENT
    WHERE pciting.POSTATE = pcited.POSTATE
      AND pciting.POSTATE IS NOT NULL
      AND pcited.POSTATE IS NOT NULL
      AND pciting.POSTATE <> ''
      AND pcited.POSTATE <> ''
    GROUP BY c.CITED
) AS cs
ON p.PATENT = cs.patent_id
ORDER BY co_state_citations DESC
LIMIT 13;


 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,co_state_citations
3845770,1974,5422,1972,US,CA,22715,2,,424,3,31,,339,,0.5152,,17.0118,,,,0.7818,0.7611,268
5111638,1992,11820,1991,US,IL,251075,2,16,53,6,68,39,254,0.4872,0.6286,0.7867,4.3307,32.0256,0.1818,0.0513,0.1633,0.1614,247
4733521,1988,10315,1986,US,IL,251075,2,16,53,6,68,24,236,0.5,0.7189,0.75,8.6737,30.9167,0,0,0.2684,0.2627,223
4773182,1988,10497,1987,US,IL,251075,2,10,47,6,61,27,225,0.5556,0.7597,0.8178,7.6444,29.4444,0.1667,0.037,0.2968,0.2889,215
4683195,1987,10070,1986,US,CA,97235,2,26,435,3,33,1,631,1,0.3464,0,7.8494,4,0,0,0.0183,0.0174,213
4418068,1983,8733,1981,US,IN,170335,2,62,514,3,31,7,225,1,0.3028,0.6939,14.1467,5.7143,0.4286,0.4286,0.9372,0.9289,208
4133814,1979,6948,1976,US,IN,170335,2,10,548,1,14,4,219,1,0.316,0.5,18.9543,2.5,0,0,0.9495,0.9452,207
3916899,1975,5786,1974,US,CA,22715,2,36,424,3,31,1,231,1,0.3987,0,15.684,1,1,1,0.8438,0.8182,198
4683202,1987,10070,1985,US,CA,97235,2,21,435,3,33,0,605,,0.3529,,8.8843,,,,0.0157,0.0149,188
4216620,1980,7529,1976,US,IL,251075,2,5,47,6,61,7,197,0,0.7048,,18.269,51.5714,0,0,0.2957,0.2792,184


In [42]:
%%sql
SELECT COUNT(*) AS total_citation_rows
FROM citations
WHERE CITED = 5959466;


 * sqlite:///patents.sq3
Done.


total_citation_rows
0


In [43]:
%%sql
SELECT COUNT(DISTINCT CITING) AS distinct_citing_patents
FROM citations
WHERE CITED = 5959466;

 * sqlite:///patents.sq3
Done.


distinct_citing_patents
0


In [44]:
%%sql
SELECT * FROM patents WHERE PATENT = 5959466;

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD
5959466,1999,14515,1997,US,CA,5310,2,,326,4,46,159,0,1,,0.6186,,4.8868,0.0455,0.044,,


In [45]:
%%sql
SELECT * FROM citations 
WHERE CITED = 5959466 OR CITING = 5959466
LIMIT 5;


 * sqlite:///patents.sq3
Done.


CITING,CITED
5959466,4195352
5959466,4458163
5959466,4527115
5959466,4609986
5959466,4631686


In [46]:
%%sql
SELECT COUNT(*) FROM citations;


 * sqlite:///patents.sq3
Done.


COUNT(*)
16522438


I am pretty confident I am using a different dataset based on some of the exploration of the data I have done above. I am turning in the code in the cell below as my final answer.

In [47]:
%%sql
SELECT p.*,
       IFNULL(cs.co_state_count, 0) AS CO_STATE_CITATIONS
FROM patents AS p
LEFT JOIN (
    SELECT c.CITED AS patent_id,
           COUNT(DISTINCT c.CITING) AS co_state_count
    FROM citations AS c
    JOIN patents AS pciting
      ON c.CITING = pciting.PATENT
    JOIN patents AS pcited
      ON c.CITED = pcited.PATENT
    WHERE pciting.POSTATE = pcited.POSTATE
      AND pciting.POSTATE IS NOT NULL
      AND pcited.POSTATE IS NOT NULL
      AND pciting.POSTATE <> ''
      AND pcited.POSTATE <> ''
    GROUP BY c.CITED
) AS cs
ON p.PATENT = cs.patent_id
ORDER BY CO_STATE_CITATIONS DESC
LIMIT 13;


 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,CO_STATE_CITATIONS
3845770,1974,5422,1972,US,CA,22715,2,,424,3,31,,339,,0.5152,,17.0118,,,,0.7818,0.7611,268
5111638,1992,11820,1991,US,IL,251075,2,16,53,6,68,39,254,0.4872,0.6286,0.7867,4.3307,32.0256,0.1818,0.0513,0.1633,0.1614,247
4733521,1988,10315,1986,US,IL,251075,2,16,53,6,68,24,236,0.5,0.7189,0.75,8.6737,30.9167,0,0,0.2684,0.2627,223
4773182,1988,10497,1987,US,IL,251075,2,10,47,6,61,27,225,0.5556,0.7597,0.8178,7.6444,29.4444,0.1667,0.037,0.2968,0.2889,215
4683195,1987,10070,1986,US,CA,97235,2,26,435,3,33,1,631,1,0.3464,0,7.8494,4,0,0,0.0183,0.0174,213
4418068,1983,8733,1981,US,IN,170335,2,62,514,3,31,7,225,1,0.3028,0.6939,14.1467,5.7143,0.4286,0.4286,0.9372,0.9289,208
4133814,1979,6948,1976,US,IN,170335,2,10,548,1,14,4,219,1,0.316,0.5,18.9543,2.5,0,0,0.9495,0.9452,207
3916899,1975,5786,1974,US,CA,22715,2,36,424,3,31,1,231,1,0.3987,0,15.684,1,1,1,0.8438,0.8182,198
4683202,1987,10070,1985,US,CA,97235,2,21,435,3,33,0,605,,0.3529,,8.8843,,,,0.0157,0.0149,188
4216620,1980,7529,1976,US,IL,251075,2,5,47,6,61,7,197,0,0.7048,,18.269,51.5714,0,0,0.2957,0.2792,184
